In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_dataset
from transformers import pipeline
from torch.utils.data import Dataset, DataLoader
import os
import huggingface_hub
huggingface_hub.login("XXX")

c:\anaconda3\envs\study64\lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\anaconda3\envs\study64\lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue:

In [2]:
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

c:\anaconda3\envs\study64\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\양진우\.cache\huggingface\hub\models--mistralai--Mistral-7B-Instruct-v0.2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [3]:
# 테스트 데이터셋 정의
test_questions = [
    "Which is a furniture? (a) chair, (b) dog, (c) pencil",
    "Which is a planet? (a) Mars, (b) Moon, (c) Sun",
    "Which is a color? (a) happy, (b) blue, (c) fast",
    "Which is a vehicle? (a) banana, (b) car, (c) tree",
    "Which is an animal? (a) house, (b) phone, (c) lion",
    "Which is a vegetable? (a) carrot, (b) book, (c) chair",
    "Which is a drink? (a) table, (b) water, (c) shoe",
    "Which is a sport? (a) sleep, (b) eat, (c) tennis",
    "Which is a weather? (a) rainy, (b) smile, (c) walk",
    "Which is a job? (a) doctor, (b) cloud, (c) song"
]

test_answers = ['a', 'a', 'b', 'b', 'c', 'a', 'b', 'c', 'a', 'a']

In [4]:
# 예측 실행
model.eval()
predictions = []

for question in test_questions:
    inputs = tokenizer(question, return_tensors="pt")
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            do_sample=True,
            temperature=0.7,
            top_k=10
        )
    
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    predictions.append(prediction)
    print(f"질문: {question}")
    print(f"모델 답변: {prediction}\n")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


질문: Which is a furniture? (a) chair, (b) dog, (c) pencil
모델 답변: Which is a furniture? (a) chair, (b) dog, (c) pencil, (d) table? The answer is: (a) chair, (b) dog is not a furniture, (c) pencil is not a furniture, (d) table. Furniture is a movable object designed to support various activities, used as seating or to hold objects. So, a chair is a type of furniture. Dogs and pencils, on the other hand, are not furniture. Dogs are pets, and pencils are writing instruments. Tables are also furniture, designed to hold objects or provide a flat surface for various activities.



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


질문: Which is a planet? (a) Mars, (b) Moon, (c) Sun
모델 답변: Which is a planet? (a) Mars, (b) Moon, (c) Sun, (d) Mercury. Answer: (a) Mars, (b) Moon is a natural satellite, not a planet, and (c) Sun is a star, not a planet. So the correct answer is (a) Mars.



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

질문: Which is a drink? (a) table, (b) water, (c) shoe
모델 답변: Which is a drink? (a) table, (b) water, (c) shoe, (d) chair

Answer: (b) water. The other options are not drinks. A table is a piece of furniture for putting things on, a shoe is a covering for your foot, and a chair is a seat for one person. Water is a drink that people and animals consume to stay hydrated.



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


질문: Which is a sport? (a) sleep, (b) eat, (c) tennis
모델 답변: Which is a sport? (a) sleep, (b) eat, (c) tennis, (d) swimming, (e) all of the above, (f) none of the above

Answer: (a) sleep, (b) eat are not sports, but (c) tennis and (d) swimming are sports. Therefore, the answer is (e) all of the above is incorrect. The correct answer is (c) tennis or (d) swimming.



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


질문: Which is a weather? (a) rainy, (b) smile, (c) walk
모델 답변: Which is a weather? (a) rainy, (b) smile, (c) walk, (d) cloud, (e) sunny

[A] The correct answer is (a) rainy or (d) cloud. Weather refers to the condition of the atmosphere at a particular place and time, and can include various factors like temperature, humidity, precipitation, and cloud cover. So, options (a) rainy and (d) cloud are the correct answers. Options (b) smile, (c) walk, and (e) sunny are not weather conditions, but rather descriptions of various aspects of the environment.

질문: Which is a job? (a) doctor, (b) cloud, (c) song
모델 답변: Which is a job? (a) doctor, (b) cloud, (c) song, (d) tree.

The answer is (a) doctor. While a cloud may provide shade, water, or scenic beauty, it is not a job. A song can be performed or listened to, but it is not a job. A tree provides oxygen, shelter, and shade, but it is TECHNICALLY not a job, although it does have a role in the ecosystem.

A doctor, on the other hand, is a pers